In [1]:
import pandas as pd
import numpy as np
import csv
import re
import string
from itertools import product

In [2]:
test_file_list_A = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\listA.csv"
test_file_repeat_list_A = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\repeat read_list A.csv"
test_file_list_B = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\listB.csv"
test_file_repeat_list_B_end = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\repeat read_list B_end.csv"
test_file_repeat_listB_96 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\repeat read_listB_96.csv"
test_file = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Protein titration LissamineFA_22267.csv"
test_file_list_C = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\Outputfiles\\csv\\listC.csv"

In [9]:
reader = csv.reader(open(test_file_repeat_list_A, 'r'), delimiter=',')   
    
all_data = [] 

for row in reader:
    all_data.append(row)   

total_lines = len(all_data)
skiprows = total_lines + 1

for index, item in enumerate(all_data):
    if item != [] and len(item) == 1 and re.findall(r"Plate information", item[0]) == ["Plate information"]:
        skiprows_meta = index + 1

    if item != [] and len(item) >= 2 and re.findall(r"PlateNumber", item[0]) == ['PlateNumber'] and re.findall(r"PlateRepeat", item[1]) == ['PlateRepeat']:   # find line number with the beggining of the data
        skiprows = index - 1
        data_format = 'A'

    if item != [] and len(item) >= 2 and re.findall(r"Plate", item[0]) == ['Plate'] and re.findall(r"Barcode", item[1]) == ['Barcode']:   # find line number with the beggining of the data
        skiprows = index
        data_format = 'B'

    if item != [] and len(item) >= 2 and re.findall(r"Plate", item[0]) == ['Plate']  and re.findall(r"Well", item[1]) == ['Well']:
        skiprows = index
        data_format = 'C'

    if item != [] and re.findall(r"G-factor", item[0]) == ["G-factor"]:
        g_factor = float(item[4])
            
    if item == [] and index > skiprows:   # find the index of the first blank row after the data table
        end_of_data = index
        break  

nrows = end_of_data - skiprows - 1
nrows_meta = (skiprows_meta + int(all_data[skiprows_meta + 1][1]) + 3) - skiprows_meta

data = pd.read_csv(test_file_repeat_list_A, sep=',', engine='python', skiprows=skiprows, nrows=nrows, encoding='utf-8')
metadata = pd.read_csv(test_file_repeat_list_A, sep=',', engine='python', skiprows=skiprows_meta, nrows=nrows_meta, encoding='utf-8')

data_frames = {}
repeats = list(metadata['Repeat'].to_numpy())

# Only for list A
for repeat in repeats:
    groupped_data = data.groupby(data.PlateRepeat).get_group(repeat)
    meta = metadata.iloc[repeat-1]
    p = groupped_data.iloc[::3, :]
    s = groupped_data.iloc[1::3, :]
               
    data_frames[f'repeat_{repeat}'] = {'metadata': meta, 'data': {'p': p, 's':s}}

In [4]:
metadata

,Plate,Repeat,Barcode,Measured height,Chamber temperature at start,Chamber temperature at end,Humidity at start,Humidity at end,Ambient temperature at start,Ambient temperature at end,Measurement date,Unnamed: 11
0,1,1,NaN,14.4,18.98,18.8,61.7,61.5,18.98,18.9,17/11/2020 13:33:43,NaN
1,1,2,NaN,14.4,18.80,18.7,61.5,61.8,18.90,19.1,17/11/2020 13:35:26,NaN


In [5]:
len(data_frames)

2

In [6]:
data_frames['repeat_1']['metadata']

Plate                                             1
Repeat                                            1
Barcode                                         NaN
Measured height                                14.4
Chamber temperature at start                  18.98
Chamber temperature at end                     18.8
Humidity at start                              61.7
Humidity at end                                61.5
Ambient temperature at start                  18.98
Ambient temperature at end                     18.9
Measurement date                17/11/2020 13:33:43
Unnamed: 11                                     NaN
Name: 0, dtype: object

In [7]:
data_frames['repeat_1']['data']['p']

,PlateNumber,PlateRepeat,Barcode,Group,Well,Label,Kinetic,ScanX,ScanY,MeasInfo,MeasTime,Signal,Flashes/Time,Result,Formula,Unnamed: 15
0,1,1,NaN,1.0,A01,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),00:00:10.511,18781491.0,60.0,18781491.0,NaN,NaN
6,1,1,NaN,1.0,A02,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),00:00:14.806,27318000.0,60.0,27318000.0,NaN,NaN
12,1,1,NaN,1.0,A03,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),00:00:19.101,15885300.0,60.0,15885300.0,NaN,NaN
18,1,1,NaN,1.0,A04,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),00:00:23.396,21036054.0,60.0,21036054.0,NaN,NaN
24,1,1,NaN,1.0,A05,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),00:00:27.682,22823211.0,60.0,22823211.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,1,1,NaN,1.0,K20,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),00:01:35.131,25057561.0,60.0,25057561.0,NaN,NaN
840,1,1,NaN,1.0,K21,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),00:01:39.418,10428058.0,60.0,10428058.0,NaN,NaN
846,1,1,NaN,1.0,K22,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),00:01:43.710,8041741.0,60.0,8041741.0,NaN,NaN
852,1,1,NaN,1.0,K23,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=1st Ex=Top Em=Top Wdw=N/A (15),00:01:48.004,13500922.0,60.0,13500922.0,NaN,NaN


In [8]:
data_frames['repeat_2']['data']['s']

,PlateNumber,PlateRepeat,Barcode,Group,Well,Label,Kinetic,ScanX,ScanY,MeasInfo,MeasTime,Signal,Flashes/Time,Result,Formula,Unnamed: 15
3,1,2,NaN,1.0,A01,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),00:01:54.075,20296635.0,60.0,20296635.0,NaN,NaN
9,1,2,NaN,1.0,A02,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),00:01:58.367,29313577.0,60.0,29313577.0,NaN,NaN
15,1,2,NaN,1.0,A03,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),00:02:02.662,18404541.0,60.0,18404541.0,NaN,NaN
21,1,2,NaN,1.0,A04,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),00:02:06.958,23410696.0,60.0,23410696.0,NaN,NaN
27,1,2,NaN,1.0,A05,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),00:02:11.245,24606892.0,60.0,24606892.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,1,2,NaN,1.0,K20,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),00:03:19.347,26424284.0,60.0,26424284.0,NaN,NaN
843,1,2,NaN,1.0,K21,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),00:03:23.634,11337218.0,60.0,11337218.0,NaN,NaN
849,1,2,NaN,1.0,K22,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),00:03:27.923,8801218.0,60.0,8801218.0,NaN,NaN
855,1,2,NaN,1.0,K23,Copy of Kris FP Fluorescein anisotropy(1),0,0,0,De=2nd Ex=Top Em=Top Wdw=N/A (142),00:03:32.218,14211489.0,60.0,14211489.0,NaN,NaN
